In [ ]:
import json
from kreis_loader import get_kreise

kreise = get_kreise(returnGeometry=True)

with open("../kreise.json", "w") as json_file:
    json.dump(kreise, json_file)

In [ ]:
from kreis_loader import get_kreise, get_envelope
from stations_loader import stations_find, filter_stations
from map_drawer import DrawMap as dm
import json

kreise = get_kreise(returnGeometry=True)

# Create an instance of DrawMap
dm = dm()

for kreis in kreise:
    station_map = dm.plot_regions(kreis)

    polygon = kreis.get("geometry", None)  # Using get() to avoid KeyError
    if polygon is not None:
        # Compute the envelope and add it to the kreis
        envelope = get_envelope(polygon)
        kreis['envelope'] = envelope
    
    stations = stations_find(geometry = envelope)
    kreis['stations'] = stations

    stations_filtered = filter_stations(polygon, stations)
    kreis['stations_filtered'] = stations_filtered
    station_map = dm.add_stations(kreis["stations_filtered"])

with open("../kreise.json", "w") as json_file:
    json.dump(kreise, json_file)

station_map

In [3]:
import json

# Open the file for reading
with open('../../kreise.json', 'r') as json_file:
    kreise = json.load(json_file)


In [18]:
import pandas as pd
from pandas import json_normalize

def create_dataframe_from_object(single_object):
    # Flattening the 'attributes', 'stations', and 'stations_filtered' fields
    df_attributes = pd.json_normalize(single_object['attributes'])
    df_stations = pd.json_normalize(single_object['stations'])
    df_stations_filtered = pd.json_normalize(single_object['stations_filtered'])

    # Aligning all DataFrames' indices before concatenating
    #df_attributes.index = [0] * len(df_attributes)
    #df_stations.index = [0] * len(df_stations)
    #df_stations_filtered.index = [0] * len(df_stations_filtered)

    # Concatenating all DataFrames horizontally
    #df_combined = pd.concat([df_attributes
                             #, [df_stations]
                             #, df_stations_filtered
                             #], axis=1)

    return df_attributes, df_stations, df_stations_filtered

df_attributes = create_dataframe_from_object(kreise[0])[0]
df_stations = create_dataframe_from_object(kreise[0])[1]
df_stations_filtered = create_dataframe_from_object(kreise[0])[2]